In [1]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import multiprocessing as mp

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

In [2]:
from wikipedia_kaggle import load_wikipedia_dataframe
wiki_df = load_wikipedia_dataframe()

In [3]:
#Dataframe that stores the keys for compact csv files
keys_df = pd.read_csv('../key_1.csv/key_1.csv')

In [4]:
#def get_pagedate_key(pagedate):
    #return keys_df.loc[keys_df['Page'] == pagedate]['Id'].values[0]

In [5]:
#get_pagedate_key("2NE1_zh.wikipedia.org_all-access_spider_2017-01-08")

In [6]:
train_date_stamps = list(wiki_df.axes[1][1:-4])

future_df = pd.read_csv('forecast_csv.csv')

In [7]:
def forecast_row(row_series):
    pagename = row_series['Page']
    row_values = row_series.drop(['Page', 'lang', 'access_type', 'agent', 'pagename']).tolist()
    
    train_df = pd.DataFrame(data={'ds':train_date_stamps, 'y':row_values})
    
    m = Prophet(yearly_seasonality=False, uncertainty_samples = 0)
    
    #try:
    m.fit(train_df) #Train classifier
    forecast = m.predict(future_df)
    
    #except TypeError:
        #If type error, mean every value is missing
        #return forecast of 0
        #forecast = pd.DataFrame(data={'ds':future_df, 'yhat':[0]*len(future_df)})

    
    forecast['yhat'] = forecast['yhat'].clip(lower=0, axis=0).round(2) #Ensure negative values go 0, round to save space
    forecast['ds'] = pd.Series(data=[pagename + "_"]*row_series.size) + forecast['ds'].map(lambda a: str(a)[:-9])
    
    return forecast[['ds','yhat']]

In [8]:
def forecast_and_save(fc_index):
    fc = forecast_row(wiki_df.iloc[fc_index])
    #Get index of the first item id
    
    first_index = keys_df.loc[keys_df['Page'] == fc['ds'][0]].index[0]
    
    def get_pagekey(k):
        return keys_dict[k]
    
    
    #Create keys dict
    keys_dict = dict(keys_df[first_index:first_index + 500].values)
    
    #try:
    fc['ds'] = fc['ds'].map(get_pagekey)
    fc.to_csv("forecasts/fc_{}.csv".format(fc_index), index=False)
    
    #except KeyError:
        #print(first_index)
        #raise Exception("sfasf")
    

In [10]:
import time

In [ ]:
with mp.Pool(1) as pool:
    start_time = time.time()
    pool.map(forecast_and_save, range(60000,60050))
    end_time = time.time() - start_time
    print(end_time)

In [12]:
from IPython.display import clear_output

#for fc_index in range(146,500):
    #clear_output(wait=True)
    #print("Working on {}...".format(fc_index))
    #forecast_and_save(fc_index)

In [13]:
#print(len(wiki_df))

In [14]:
pool = mp.Pool(4) # use 4 processes

In [15]:
#from ThreadPool import ThreadPool

In [16]:
#pool = ThreadPool(100)

In [19]:
def forecast_and_save_range(start, stop):
    for fc_index in range(start, stop):
        clear_output(wait=True)
        print("Working on {}...".format(fc_index))
        try:
            forecast_and_save(fc_index)
        except:
            print("ERROR ON {}".format(fc_index))

In [24]:
for s in range(50000, 51000, 200):
    print(type(forecast_and_save_range))
    start = s
    stop = s + 500
    #pool.apply_async(forecast_and_save_range,[start, stop])
    pool.add_task(forecast_and_save_range, start, stop)

<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>


In [23]:
#pool.wait_completion()

In [ ]:
print("DONE!")